In [3]:
import wtss
import pandas as pd
import geopandas as gpd

import numpy as np

import tqdm

In [24]:
rng = np.random.default_rng(20241217)

In [22]:

pontos_path =  'points_20x20.shp'

In [26]:
pontos = gpd.read_file(pontos_path)

In [27]:
pontos.head()

,x,y,inside,geometry
0,-48.089043,-15.677226,0,POINT (-48.08904 -15.67723)
1,-48.089043,-15.675568,0,POINT (-48.08904 -15.67557)
2,-48.089043,-15.673910,0,POINT (-48.08904 -15.67391)
3,-48.089043,-15.672252,0,POINT (-48.08904 -15.67225)
4,-48.089043,-15.670594,0,POINT (-48.08904 -15.67059)


In [5]:
pontos.head()

,left,top,right,bottom,Hectares,din_class,din_classF,id_2,id_ponto,geometry
0,-47.378595,-16.784793,-47.377925,-16.784123,75.606338,WC,SAFRA DUPLA (INVERNO),0,0,POINT (-47.3786 -16.78479)
1,-47.378595,-16.785463,-47.377925,-16.784793,75.606338,WC,SAFRA DUPLA (INVERNO),0,1,POINT (-47.3786 -16.78546)
2,-47.378595,-16.786133,-47.377925,-16.785463,75.606338,WC,SAFRA DUPLA (INVERNO),0,2,POINT (-47.3786 -16.78613)
3,-47.378595,-16.786803,-47.377925,-16.786133,75.606338,WC,SAFRA DUPLA (INVERNO),0,3,POINT (-47.3786 -16.7868)
4,-47.378595,-16.787473,-47.377925,-16.786803,75.606338,WC,SAFRA DUPLA (INVERNO),0,4,POINT (-47.3786 -16.78747)


In [28]:
pontos = pontos.groupby('inside', group_keys=False).apply(lambda x: x.sample(
    frac=0.20, random_state=rng))

In [29]:
servico = wtss.WTSS('https://data.inpe.br/bdc/wtss/v4/')
servico.coverages

['CBERS4-MUX-2M-1',
 'CBERS4-WFI-16D-2',
 'CBERS-WFI-8D-1',
 'LANDSAT-16D-1',
 'mod13q1-6.1',
 'myd13q1-6.1',
 'S2-16D-2']

In [30]:
cubo_s2 = servico['S2-16D-2']

In [31]:
timeline = cubo_s2.timeline

ultima_data_disponivel = timeline[0]
primeira_data_disponivel = timeline[-1]

print(f'Intervalo: [{primeira_data_disponivel}, {ultima_data_disponivel}]')

Intervalo: [2024-07-27, 2017-01-01]


In [32]:
start_date = '2020-06-01'
end_date = '2022-09-30'

attributes = ('NDVI', 'SCL')

In [33]:


ndvi_values = []
scl_values = []

for id, geometry in tqdm.tqdm(zip(pontos.inside.values,
                                  pontos.geometry.values),
                         desc="Processando dados", unit="data"):

    timeseries = cubo_s2.ts(attributes=attributes,
                            geom=geometry,
                            start_date=start_date,
                            end_date=end_date)
    
    tmp = timeseries.df('NDVI')   

    ndvi_values.append(tmp.loc[tmp.attribute == 'NDVI', 'value'])
    scl_values.append(tmp.loc[tmp.attribute == 'SCL', 'value'])



Processando dados: 520data [03:56,  2.20data/s]


In [34]:
ndvi_values

[0    -0.5849
 1    -0.3739
 2    -0.2745
 3    -0.2807
 4    -0.1224
 5    -0.1236
 6    -0.0332
 7     0.0068
 8     0.0273
 9     0.0401
 10   -0.0384
 11   -0.0474
 12    0.0622
 13    0.0340
 14    0.0021
 15   -0.0515
 16    0.0208
 17   -0.0590
 18   -0.0954
 19   -0.1241
 20   -0.1212
 21    0.0480
 22   -0.2656
 23   -0.3281
 24   -0.5505
 25   -0.3962
 26   -0.5434
 27   -0.1978
 28   -0.1705
 29   -0.0522
 30   -0.1081
 31    0.0077
 32    0.0378
 33   -0.0519
 34    0.1731
 35   -0.0414
 36    0.0016
 37   -0.0705
 38   -0.0698
 39    0.0044
 40    0.0592
 41   -0.0928
 42    0.0171
 43   -0.0487
 44   -0.1338
 45   -0.2540
 46   -0.3006
 47   -0.2081
 48   -0.1958
 49   -0.2868
 50   -0.1967
 51   -0.1357
 52   -0.0379
 53   -0.0090
 54    0.0247
 Name: value, dtype: float64,
 0     0.6993
 1     0.6536
 2     0.6215
 3     0.5957
 4     0.5664
 5     0.5235
 6     0.4487
 7     0.4469
 8     0.4845
 9     0.0234
 10    0.1398
 11    0.7401
 12    0.6249
 13    0.6760
 14 

In [35]:
ndvi = pd.DataFrame(data = np.asarray(ndvi_values).T,
                    columns=pontos.inside.values,
                    index=timeseries.ts.timeline)

In [36]:
ndvi = ndvi.reset_index().rename(columns={'index': 'data'})

In [37]:
ndvi.to_csv( 'ndvi_sample_v3.csv')

In [38]:
scl = pd.DataFrame(data = np.asarray(scl_values).T,
                    columns=pontos.inside.values,
                    index=timeseries.ts.timeline)

scl = scl.reset_index().rename(columns={'index': 'data'})


In [40]:
scl.to_csv( 'scl_sample_v3.csv')